In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import pandas as pd
from branca.colormap import linear
from DadosAbertosBrasil import ibge
import statsmodels.api as sm
from branca.colormap import LinearColormap
import warnings
warnings.filterwarnings("ignore")

/Users/juliasbardelatti/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df = pd.read_csv("analise_geral.csv")
df = df.query("ano == 2014 or ano == 2024")
df.head()

,Unnamed: 0,regiao,codigo_uf,uf,codigo_ibge,municipio,qtd_magreza_acentuada,perc_magreza_acentuada,qtd_magreza,perc_magreza,qtd_eutrofia,perc_eutrofia,qtd_risco_sobrepeso,perc_risco_sobrepeso,qtd_sobrepeso,perc_sobrepeso,qtd_obesidade,perc_obesidade,total,ano
0,0,CENTRO-OESTE,53.0,DF,530010.0,BRASILIA,615.0,0.0256,662.0,0.0276,15.887,0.6621,4.041,0.1684,1.421,0.0592,1.369,0.0571,23.995,2014
1,1,TOTAL ESTADO DISTRITO FEDERAL,NaN,NaN,NaN,NaN,615.0,0.0256,662.0,0.0276,15.887,0.6621,4.041,0.1684,1.421,0.0592,1.369,0.0571,23.995,2014
2,2,CENTRO-OESTE,52.0,GO,520005.0,ABADIA DE GOIAS,6.0,0.0508,4.0,0.0339,48,0.4068,21.0,0.1780,19.000,0.1610,20.000,0.1695,118,2014
3,3,CENTRO-OESTE,52.0,GO,520010.0,ABADIANIA,9.0,0.0474,10.0,0.0526,105,0.5526,23.0,0.1211,24.000,0.1263,19.000,0.1000,190,2014
4,4,CENTRO-OESTE,52.0,GO,520013.0,ACREUNA,34.0,0.0771,34.0,0.0771,234,0.5306,61.0,0.1383,32.000,0.0726,46.000,0.1043,441,2014


In [3]:
df_estado = df[df['regiao'].str.startswith(('TOTAL ESTADO'), na=False)]
df_estado['NOME'] = df_estado['regiao'].str.replace('TOTAL ESTADO ', '').str.strip()

base_codigos = pd.read_csv("estados.csv")
base_codigos['NOME'] = base_codigos['NOME'].str.upper()
resultado = df_estado.merge(base_codigos, on='NOME', how='left')

colunas_para_converter = ['perc_magreza_acentuada', 'perc_magreza', 'perc_eutrofia', 'perc_risco_sobrepeso', 'perc_sobrepeso', 'perc_obesidade'] 

for coluna in colunas_para_converter:
    resultado[coluna] = pd.to_numeric(resultado[coluna], errors='coerce') 

resultado['excesso_de_peso'] = (resultado['perc_risco_sobrepeso'] + resultado['perc_sobrepeso'] + resultado['perc_obesidade']) 

resultado = resultado[['ano', 'NOME', 'COD', 'SIGLA', 'excesso_de_peso']]

In [4]:
resultado['ano'] = resultado['ano'].astype(int)

df_2014 = resultado[resultado['ano'] == 2014].set_index(['COD', 'SIGLA', 'NOME'])
df_2024 = resultado[resultado['ano'] == 2024].set_index(['COD', 'SIGLA', 'NOME'])

df_comparado = df_2014[['excesso_de_peso']].join(
    df_2024[['excesso_de_peso']], lsuffix='_2014', rsuffix='_2024'
)

df_comparado['variacao_absoluta'] = (df_comparado['excesso_de_peso_2024'] - df_comparado['excesso_de_peso_2014'])
df_comparado = df_comparado.reset_index()

In [5]:
df_comparado.sort_values("variacao_absoluta").to_clipboard(decimal=",")

In [6]:
geojson_path = ibge.malha(
    nivel = 'paises',
    localidade = 'BR',
    divisoes = 'uf',
    formato = 'geojson'
)

In [8]:
mapa = folium.Map(location=[-14.2350, -51.9253], zoom_start=4, tiles=None)

if isinstance(geojson_path, dict):
    gdf = gpd.GeoDataFrame.from_features(geojson_path["features"])
else:
    gdf = gpd.read_file(geojson_path)

gdf["codarea"] = gdf["codarea"].astype(int)
siglas = {
    11: "RO", 12: "AC", 13: "AM", 14: "RR", 15: "PA", 16: "AP", 17: "TO",
    21: "MA", 22: "PI", 23: "CE", 24: "RN", 25: "PB", 26: "PE", 27: "AL", 28: "SE", 29: "BA",
    31: "MG", 32: "ES", 33: "RJ", 35: "SP",
    41: "PR", 42: "SC", 43: "RS",
    50: "MS", 51: "MT", 52: "GO", 53: "DF"
}
gdf["sigla"] = gdf["codarea"].map(siglas)
df_comparado["variacao_absoluta"] = df_comparado["variacao_absoluta"] * 100
min_val = df_comparado["variacao_absoluta"].min()
max_val = df_comparado["variacao_absoluta"].max()

colormap = LinearColormap(
    colors=["#08306b", "#deebf7"],
    vmin=min_val,
    vmax=max_val,
    caption="Variação em pontos percentuais"
)
colormap.add_to(mapa) 

def style_function(feature):
    cod = int(feature["properties"]["codarea"])
    row = df_comparado[df_comparado["COD"] == cod]
    if not row.empty:
        variacao = row.iloc[0]["variacao_absoluta"]
        color = colormap(variacao)
    else:
        color = "white"  # Quando não há dados, mantém o preenchimento branco

    return {
        "fillColor": color,
        "color": "black",  # Aumenta o contraste da borda para "black"
        "weight": 0.8,  # Aumenta o peso da borda
        "fillOpacity": 0.7,
        "opacity": 0.8,  # Aumenta a opacidade para garantir visibilidade
    }

folium.GeoJson(
    geojson_path,
    style_function=style_function,
    name="Variação"
).add_to(mapa)

for _, row in gdf.iterrows():
    estado_codigo = row["codarea"]
    centroide = row.geometry.centroid
    variacao_absoluta = df_comparado.loc[df_comparado["COD"] == estado_codigo, "variacao_absoluta"]
    
    if not variacao_absoluta.empty:
        variacao_valor = round(variacao_absoluta.values[0], 0)
        sigla_estado = row["sigla"]
        sinal = "+" if variacao_valor > 0 else ""
        texto_variacao = f"{sinal}{int(variacao_valor)}pp"

        if sigla_estado == "RN":
            centroide_y = centroide.y + 0.4
            centroide_x = centroide.x
        elif sigla_estado == "AL":
            centroide_y = centroide.y
            centroide_x = centroide.x + 0.8  
        elif sigla_estado == "PB":
            centroide_y = centroide.y
            centroide_x = centroide.x + 0.8  
        else:
            centroide_y = centroide.y
            centroide_x = centroide.x

        destaque = abs(variacao_valor) >= 10
        cor_texto = "black" if destaque else "black"
        tamanho = "13px" if destaque else "13px"

        folium.Marker(
            location=[centroide_y, centroide_x],
            icon=folium.DivIcon(html=f"""
                <div style="font-size: {tamanho}; font-weight: bold; color: {cor_texto}; 
                            background-color: rgba(255, 255, 255, 0.9); 
                            padding: 2px 6px; border-radius: 6px; 
                            min-width: 50px; text-align: center;">
                    {sigla_estado}<br>{texto_variacao}
                </div>
            """)
        ).add_to(mapa)
titulo_html = """
<div style="
    position: fixed; 
    top: 10px; left: 50%; 
    transform: translateX(-50%);
    z-index: 9999; 
    background-color: white; 
    padding: 10px; 
    border-radius: 8px;
    font-size: 18px; 
    font-weight: bold;
    box-shadow: 0 0 5px rgba(0,0,0,0.2);
">
        Variação em pontos percentuais do excesso de peso em crianças menores de 5 anos no Brasil (2014–2024)

</div>
"""
mapa.get_root().html.add_child(folium.Element(titulo_html))
mapa.get_root().html.add_child(folium.Element("""
    <style>
        .legend {
            position: fixed !important;
            top: 50px !important;
            left: 50% !important;
            transform: translateX(-50%);
            z-index: 9998;
        }
    </style>
"""))
mapa.get_root().html.add_child(folium.Element("""
    <style>
        html, body {
            background-color: white !important;
            height: 100%;
            margin: 0;
        }
        .folium-map { background: white; }
        #map { height: 100%; }
    </style>
"""))

mapa.save("mapa_com_siglas.html")